# ![picture](https://prowly-uploads.s3.eu-west-1.amazonaws.com/uploads/4626/assets/71776/large_logo_wsb_poziom.jpg)

# Plan zajęć
1.   Metryki błędów - modele klasyfikacyjne
2.   Metryki błędów - modele regresyjne


In [ ]:
# importowanie niezbędnych bibliotek
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
rng = np.random.RandomState(42)

Dokumentacja : https://scikit-learn.org/stable/modules/model_evaluation.html

# Regresja prosta i wieloraka

In [ ]:
num_samples = 400
mu = np.array([5.0, 0.0, 10.0])
r = np.array([
        [  3.40, 3.75, 3.00],
        [ 3.75,  5.50,  2.50],
        [ 2.00,  2.50,  2.25]
    ])
vals = np.random.multivariate_normal(mu, r, size=num_samples)
wiek_pacjenta = vals[:,1].reshape([len(vals),1])
wiek_pacjenta_doswiadczenie_lekarza = vals[:,[0,1]].reshape([len(vals),2])
czas_trwania_operacji = vals[:,2].reshape([len(vals),1])

<ipython-input-2-e621f4b6b74b>:8: RuntimeWarning: covariance is not positive-semidefinite.
  vals = np.random.multivariate_normal(mu, r, size=num_samples)


Testy statystyczne pomagają nam odpowiedzieć na pytanie, czy między wartościami istnieją zalezności. Modele regresyjne mówią o tym, w jaki sposób zmiana wartości zmiennej niezależnej wpłynie na zmienną zależną.<br>
Pomysł jest stosunkowo prosty - każda obserwacja jest opisana przez 2 wartości (nazwijmy x i y). 

In [ ]:
fig = go.Figure(
    data=[
          go.Scatter(
              x=wiek_pacjenta.flatten(),
              y=czas_trwania_operacji.flatten(),
              mode='markers'),
          ]
          )
fig.show()

Na podstawie wartości x chcemy przewidzieć, jaka będzie wartość y. W praktyce polega to na wyznaczeniu takiej kreski, która będzie (średnio) najbliżej każdej kropki.

I z matematyki pamiętamy, że wzór na funkcję liniową wygląda:<br>
<center><p>y=ax+b</p></center><p>
Współczynnik <i>a</i> odpowiada za pochylenie krzywej, zaś <i>b</i> za punkt przecięcia z osią x. W interpretacji <i>a</i> wielokrotnością wzrostu y przy wzroście x o jednostkę.

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
reg = LinearRegression().fit(wiek_pacjenta, czas_trwania_operacji)

In [ ]:
czas_trwania_operacji_predicted = reg.predict(wiek_pacjenta)

In [ ]:
fig = go.Figure(
    data=[
          go.Scatter(
              x=wiek_pacjenta.flatten(),
              y=czas_trwania_operacji.flatten(),
              mode='markers'),
          go.Scatter(
              x=wiek_pacjenta.flatten(),
              y=czas_trwania_operacji_predicted.flatten()),
          ]
          )
fig.show()

In [ ]:
print(f'Współczynnik a: {reg.coef_[0][0]}, współczynnik b(wyraz wolny): {reg.intercept_[0]}')
print(f'Interpretacja: jeżeli wiek pacjenta wzrośnie o 10, to przewidywany czas operacji wzrasta o {10*reg.coef_[0][0]}')

Współczynnik a: 0.5063401551716475, współczynnik b(wyraz wolny): 10.141331953267969
Interpretacja: jeżeli wiek pacjenta wzrośnie o 10, to przewidywany czas operacji wzrasta o 5.063401551716476


W przypadku regresji wielorakiej, pomysł jest dokładnie taki sam - różni się jedynie tym, że zamiast jednej zmiennej niezależnej (x) mamy ich więcej (np. dwie - x<sub>1</sub> i x<sub>2</sub>). W tym przypadku wzór funkcji wygląda następujaco: <br>
<center><p>y=ax<sub>1</sub>+bx<sub>2</sub>+c</p></center><p>
Interpretacja współczynników jest analogiczna jak w przypadku modelu z jedną zmienną.

In [ ]:
reg = LinearRegression().fit(wiek_pacjenta_doswiadczenie_lekarza, czas_trwania_operacji)
linspace_x = np.linspace(2,10,100)
linspace_x = np.array([np.repeat(x,100) for x in linspace_x]).flatten()
linspace_y = np.tile(np.linspace(-8,6,100),100)
czas_trwania_operacji_predicted = reg.predict(np.c_[linspace_x,linspace_y])
fig = go.Figure(
    data=[
          go.Scatter3d(
              x=wiek_pacjenta_doswiadczenie_lekarza[:,0].flatten(),
              y=wiek_pacjenta_doswiadczenie_lekarza[:,1].flatten(),
              z=czas_trwania_operacji.flatten(),
              mode='markers',
              marker=dict(
                  size = 4
              )),
          go.Scatter3d(
              x=linspace_x,
              y=linspace_y,
              z=czas_trwania_operacji_predicted.flatten(),
              mode='lines',
              marker=dict(
                  size = 5
              )),
          ]
          )
fig.show()

In [ ]:
print(f'Współczynnik a (stoi przy wieku pacjenta): {reg.coef_[0][0]}, współczynnik b (stoi przy doświadczeniu lekarza): {reg.coef_[0][1]}, współczynnik c (wyraz wolny): {reg.intercept_[0]}')
print(f'Interpretacja: jeżeli wiek pacjenta wzrośnie o 10, to przewidywany czas operacji wzrasta o {10*reg.coef_[0][0]} przy założeniu, że wartości pozostałych zmiennych są stałe')
print(f'Interpretacja: jeżeli doświadczenie lekarza wzrośnie o 10, to przewidywany czas operacji maleje o {10*abs(reg.coef_[0][1])} przy założeniu, że wartości pozostałych zmiennych są stałe')

Współczynnik a (stoi przy wieku pacjenta): 0.9053663710718137, współczynnik b (stoi przy doświadczeniu lekarza): -0.13070296550442095, współczynnik c (wyraz wolny): 5.474950334882222
Interpretacja: jeżeli wiek pacjenta wzrośnie o 10, to przewidywany czas operacji wzrasta o 9.053663710718137 przy założeniu, że wartości pozostałych zmiennych są stałe
Interpretacja: jeżeli doświadczenie lekarza wzrośnie o 10, to przewidywany czas operacji maleje o 1.3070296550442095 przy założeniu, że wartości pozostałych zmiennych są stałe


# Miary dla modeli regresyjnych

In [ ]:
from sklearn.metrics import mean_squared_error, median_absolute_error, r2_score, mean_absolute_error, mean_absolute_percentage_error

\begin{align}
MSE = \frac{\sum_{i=1}^n (y_i - \hat{y_i})^2}{N}
\end{align}

\begin{align}
RMSE = \sqrt{MSE}
\end{align}

\begin{align}
MedianAE = median(y_i - \hat{y_i})
\end{align}

\begin{align}
MAE = \frac{\sum_{i=1}^n |y_i - \hat{y_i}|}{N}
\end{align}

\begin{align}
MAPE = \sum_{i=1}^n \frac{|y_i - \hat{y_i|}}{|y_i|}
\end{align}

\begin{align}
R^2 = ?
\end{align}

Ważne! Miara MSE może zostać zdekomponowana jako suma obiążenia, wariancji i losowego, nieredukowalnego błędu. W przypadku modelowania zachodzi tak zwany bias-variance tradeoff. Doczytać!

# ![picture](https://wikimedia.org/api/rest_v1/media/math/render/svg/2ac524a751828f971013e1297a33ca1cc4c38cd6)

I tutaj warto zrozumieć, co to znaczy - część z wariancją mówi nam o tym, jak wartości estymowane średnio różnią się od średniej modelu. Wskazuje to na dopasowanie modelu do danych - przy czym zbyt wysokie dopasowanie spowoduje niski stopień generalizowania problemu, czyli przetrenowanie modelu. <br> Obciążenie (bias) wskazuje zaś na różnicę między średnią wartością predykcji a wartościami rzeczywistymi, które staramy się przewidzieć.

In [ ]:
y = czas_trwania_operacji
y_hat = reg.predict(wiek_pacjenta_doswiadczenie_lekarza)

In [ ]:
mse = mean_squared_error(y_true=y, y_pred = y_hat)
rmse = mean_squared_error(y_true=y, y_pred = y_hat, squared = False)
median_ae = median_absolute_error(y_true=y, y_pred = y_hat)
r2 = r2_score(y_true=y, y_pred = y_hat)
mae = mean_absolute_error(y_true=y, y_pred = y_hat) 
mape = mean_absolute_percentage_error(y_true=y, y_pred = y_hat)

In [ ]:
print(f'MSE={mse}, rmse={rmse}, median_ae={median_ae},r2={r2}, mae = {mae}, mape = {mape}')

MSE=0.8020389494218056, rmse=0.8955662730484024, median_ae=0.592144336192189,r2=0.750828049593309, mae = 0.7047404554012024, mape = 0.0721304730205869


In [ ]:
def sinusoida(npoints,random_state=1):
    np.random.seed(random_state)
    vec = np.zeros((npoints,2))
    vec[:,0] = np.arange(-2*np.pi,2*np.pi,4*np.pi/npoints)
    vec[:,1] = np.sin(vec[:,0])+0.1*rng.randn(npoints,1)[:,0]
    vec[:,0] = (vec[:,0]-min(vec[:,0]))/(max(vec[:,0]-min(vec[:,0])))
    vec[:,1] = (vec[:,1]-min(vec[:,1]))/(max(vec[:,1]-min(vec[:,1])))
    return(vec)

In [ ]:
sinus = sinusoida(1000,1)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=sinus[:, 0], y=sinus[:, 1],
                    mode='markers',
                    name='Wartości rzeczywiste'))
fig.show()

In [ ]:
prediction_model1 = np.repeat(0.5,1000)

In [ ]:
prediction_model2 = np.linspace(0,1,1000)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rfr = RandomForestRegressor()

In [ ]:
rfr.fit(X= sinus[:, 0].reshape(-1, 1), y = sinus[:, 1])

RandomForestRegressor()

In [ ]:
prediction_model3 = rfr.predict(sinus[:, 0].reshape(-1, 1))

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=sinus[:, 0], y=sinus[:, 1],
                    mode='markers',
                    name='Wartości rzeczywiste'))
fig.add_trace(go.Scatter(x=sinus[:, 0], y=prediction_model1.reshape(1000),
            mode='markers',
            name='Predykcje model 1'))
fig.add_trace(go.Scatter(x=sinus[:, 0], y=prediction_model2.reshape(1000),
            mode='markers',
            name='Predykcje model 2'))
fig.add_trace(go.Scatter(x=sinus[:, 0], y=prediction_model3.reshape(1000),
            mode='markers',
            name='Predykcje model 3'))

fig.show()

## Zadanie - oblicz metryki błędów dla 3 modeli
Czy model 3 jest wytrenowany optymalnie pod kątem bias-variance tradeoff?

# Modele klasyfikacyjne

W przypadku problemu klasyfikacji, zmienna y jest wyrażona na skali nominalnej - więc celem jest dopasowanie odpowiedniej etykiety do obiektu. Przykłady - klasyfikacja zdjęć, detekcja oszustw itp.

W przypadku klasyfikacji ostateczną wartością jest etykieta, jednak modele zwracają wynik w postaci wartości z pewnego przedziału - które mogą być interpretowane jako miara prawdopodobieństwa przynależnośći do danej klasy. W przypadku klasyfikacji binarnej, model zwróci wartości z przedziało 0 - 1, gdzie 0 świadczy o niskim prawdopodobieństwie przynależności do klasy oznaczonej jako 1.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from sklearn.datasets import make_classification

In [ ]:
weights = (0.8, 0.1)
class_sep = 0.8
X, y = make_classification(n_samples=1000, n_features=2, n_informative=2, 
                           n_redundant=0, n_repeated=0, n_classes=2, 
                           n_clusters_per_class=1, weights=weights, 
                           class_sep=class_sep, random_state=0)

In [ ]:
X

array([[-0.2002285 , -1.12688039],
       [-0.87362998,  0.99925873],
       [-0.72295314,  0.10616655],
       ...,
       [-2.17673965,  2.83728951],
       [-0.87726236,  1.25397626],
       [ 1.18028997,  0.67341245]])

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=X[:, 0], y=X[:, 1],
                         marker = dict(
                             color = y
                         ),
                        mode='markers',
                        name='Wartości rzeczywiste'))
fig.show()

In [ ]:
rfc = RandomForestClassifier(max_depth = 3, n_estimators=100)

In [ ]:
rfc.fit(X= X, y = y.ravel())

RandomForestClassifier(max_depth=3)

In [ ]:
predictions_classification = rfc.predict(X)

In [ ]:
predictions_classification

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
predictions_classification_proba = rfc.predict_proba(X)

In [ ]:
predictions_classification_proba[:10,1]

array([0.06268841, 0.04576931, 0.059678  , 0.04469029, 0.02427743,
       0.02427743, 0.78482732, 0.07312368, 0.05685203, 0.03621904])

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=X[:, 0], y=X[:, 1],
                         marker = dict(
                             color = predictions_classification_proba[:,1]
                         ),
                        mode='markers',
                        name='Wartości rzeczywiste'))
fig.show()

# Miary dla modeli klasyfikacyjnych

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

Pierwszą informacją, którą należy przeanalizować jest macierz błędów - czyli w ilu przypadkach poprawnie sklasyfikowaliśmy obiekty, a w ilu popełniliśmy błąd - i jaki był to błąd.

In [ ]:
pd.DataFrame(confusion_matrix(y, predictions_classification), 
             columns=['Predicted Negative', 'Predicted Positive'], 
             index=['Actual Negative', 'Actual Positive'])

,Predicted Negative,Predicted Positive
Actual Negative,843,3
Actual Positive,56,98


Na podstawie macierzy błędów wyliczamy miary jakości modelu, czyli:

Dokładkość(Accuracy) = trafione / wszystkie

Precyzja(precision) = dobrze trafione pozytywne / przewidywane pozytywne

Czułość(Recall) = dobrze trafione pozytywne / rzeczywiste pozytywne

F1 = 2*(precision * recall) / (precision + recall)

Do zapamiętania:
https://towardsdatascience.com/accuracy-precision-and-recall-never-forget-again-33e64635780 

In [ ]:
import IPython
IPython.display.IFrame(src='https://en.wikipedia.org/wiki/Template:Diagnostic_testing_diagram', width=1500, height=700)


# ![picture](https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/Precisionrecall.svg/800px-Precisionrecall.svg.png)

In [ ]:
print(
    classification_report(y_true = y, y_pred = predictions_classification)
)

              precision    recall  f1-score   support

           0       0.94      1.00      0.97       846
           1       0.97      0.64      0.77       154

    accuracy                           0.94      1000
   macro avg       0.95      0.82      0.87      1000
weighted avg       0.94      0.94      0.94      1000



Krzywa <b>ROC</b> - wykres używany do oceny poprawności modelu klasyfikacyjnego. Pokazuje skuteczność modelu, czyli jak często model podejmuje poprawne decyzje. Im większe pole pod jego powierzchnią, tym lepszy model.<br>
Wskaźnik <b>AUC</b> pokazuje, jakie jest pole pod krzywą <b>ROC</b>. Wartości wskaźnika znajdują się w przedziale [0,1], gdzie 1 - idealny model, 0.5 - wyniki modelu są takie same jak w przypadku ślepego losowania wyników.

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

In [ ]:
fpr, tpr, _ = roc_curve(y, predictions_classification_proba[:,1])
print(f'Wskaźnik AUC: {roc_auc_score(y, predictions_classification_proba[:,1])}')

Wskaźnik AUC: 0.9625472045684812


In [ ]:
print("Krzywa ROC:")
fig = go.Figure(
    data=[
          go.Scatter(
              x=fpr,
              y=tpr,
              name = 'Krzywa ROC'),
          go.Scatter(
              x=[0,1],
              y=[0,1],
              name = 'Model naiwny'),
          ]
          )
fig.update_layout(
    xaxis = dict(
        title = 'FPR = 1 - specificity'
    ),
    yaxis = dict(
        title = 'TPR = sensitivity = recall = czułość'
    )
)
fig.show()

Krzywa ROC:


Uwaga - miara ACC może być bardzo myląca w przypadku niezbalansowanych danych. Dlaczego?

## Zadanie 2 - oblicz metryki błędów dla podanego zbioru korzystając z modelu RandomForestClassifier

In [ ]:
weights = (0.9, 0.1)
class_sep = 0.1
X, y = make_classification(n_samples=1000, n_features=10, n_informative=6, 
                           n_redundant=1, n_repeated=3, n_classes=2, 
                           n_clusters_per_class=1, weights=weights, 
                           class_sep=class_sep, random_state=0)

In [ ]:
rfc = RandomForestClassifier(max_depth = 3, n_estimators=100, max_features=0.2)

In [ ]:
rfc.fit(X= X, y = y.ravel())

RandomForestClassifier(max_depth=3, max_features=0.2)

In [ ]:
predictions_classification = rfc.predict(X)